In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from scipy.optimize import curve_fit

from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [2]:
LS133 = "./ntupleStubsBmtfGmt_LS133.root"
LS263 = "./ntupleStubsBmtfGmt_LS263.root"
LS264 = "./ntupleStubsBmtfGmt_LS264.root"
LS306 = "./ntupleStubsBmtfGmt_LS306.root"

#Open file
stubs_awk_LS133 = OpenRootFile(LS133, stub_features)
stubs_awk_LS263 = OpenRootFile(LS263, stub_features)
stubs_awk_LS264 = OpenRootFile(LS264, stub_features)
stubs_awk_LS306 = OpenRootFile(LS306, stub_features)

gmt_awk_LS133 = OpenRootFile(LS133, gmt_features)
gmt_awk_LS263 = OpenRootFile(LS263, gmt_features)
gmt_awk_LS264 = OpenRootFile(LS264, gmt_features)
gmt_awk_LS306 = OpenRootFile(LS306, gmt_features)

bmtf_awk_LS133 = OpenRootFile(LS133, bmtf_features)
bmtf_awk_LS263 = OpenRootFile(LS263, bmtf_features)
bmtf_awk_LS264 = OpenRootFile(LS264, bmtf_features)
bmtf_awk_LS306 = OpenRootFile(LS306, bmtf_features)

In [16]:
#Caricamenot BX delle stubs
BX_LS133 = stubs_awk_LS133.bx
BX_LS263 = stubs_awk_LS263.bx
BX_LS264 = stubs_awk_LS264.bx
BX_LS306 = stubs_awk_LS306.bx

#Caricamento muoni di GMT
nGmtMuons_LS133 = gmt_awk_LS133.nGmtMuons
nGmtMuons_LS263 = gmt_awk_LS263.nGmtMuons
nGmtMuons_LS264 = gmt_awk_LS264.nGmtMuons
nGmtMuons_LS306 = gmt_awk_LS306.nGmtMuons

nBmtfMuons_LS133 = bmtf_awk_LS133.nBmtfMuons
nBmtfMuons_LS263 = bmtf_awk_LS263.nBmtfMuons
nBmtfMuons_LS264 = bmtf_awk_LS264.nBmtfMuons
nBmtfMuons_LS306 = bmtf_awk_LS306.nBmtfMuons

In [137]:
#GMT

weightsNGmt = ak.where(nBmtfMuons_LS133 <= 1, nBmtfMuons_LS133, 1)

bx_gmt_counts_LS133 = np.bincount(BX_LS133, weights=weightsNGmt)
bx_gmt_counts_LS263 = np.bincount(BX_LS263, weights=nGmtMuons_LS263)
bx_gmt_counts_LS264 = np.bincount(BX_LS264, weights=nGmtMuons_LS264)
bx_gmt_counts_LS306 = np.bincount(BX_LS306, weights=nGmtMuons_LS306)

#bx_gmt_counts = (bx_gmt_counts_LS133 + bx_gmt_counts_LS263 + bx_gmt_counts_LS264 + bx_gmt_counts_LS306)/4


#BMTF
weightsN = ak.where(nBmtfMuons_LS133 <= 1, nBmtfMuons_LS133, 1)


bx_bmtf_counts_LS133N = np.bincount(BX_LS133, weights=weightsN)
bx_bmtf_counts_LS133 = np.bincount(BX_LS133, weights=nBmtfMuons_LS133)

bx_bmtf_counts_LS263 = np.bincount(BX_LS263, weights=nBmtfMuons_LS263)
bx_bmtf_counts_LS264 = np.bincount(BX_LS264, weights=nBmtfMuons_LS264)
bx_bmtf_counts_LS306 = np.bincount(BX_LS306, weights=nBmtfMuons_LS306)

#bx_bmtf_counts = (bx_bmtf_counts_LS133 + bx_bmtf_counts_LS263 + bx_bmtf_counts_LS264 + bx_bmtf_counts_LS306)/4

print(bx_bmtf_counts_LS133N[100])
print(bx_bmtf_counts_LS133[100])
print(bx_gmt_counts_LS133[100])


193.0
204.0
193.0


In [111]:
def BX_OPT(BX, nMuons):

    BX_Matrix = [[] for _ in range(ONE_ORBIT_IN_BX)]
    
    for j in range(len(BX)):
      if nMuons[j] == 1:
        BX_Matrix[BX[j]].append([j])
      
      if nMuons[j] > 1:
        BX_Matrix[BX[j]].append([j, nMuons[j]])
    
    BX_Matrix = ak.Array(BX_Matrix)

    return BX_Matrix

In [122]:
BX_BMTF_LS133 = BX_OPT(BX_LS133, nBmtfMuons_LS133)
BX_GMT_LS133 = BX_OPT(BX_LS133, nGmtMuons_LS133)

In [113]:
print(BX_BMTF_LS133)

[[], [], [], [], [[1017253]], [[1648233]], [...], ..., [], [], [], [], [], []]


In [114]:
print(len(ak.flatten(BX_BMTF_LS133)))

for i in range(100):
  print(i, " ", BX_BMTF_LS133[i])

524219
0   []
1   []
2   []
3   []
4   [[1017253]]
5   [[1648233]]
6   [[273774], [1648234]]
7   [[540931], [1648235]]
8   [[1205576]]
9   []
10   [[208712]]
11   []
12   [[152325]]
13   []
14   []
15   [[1607009]]
16   []
17   []
18   []
19   []
20   []
21   []
22   []
23   []
24   []
25   [[1556071]]
26   []
27   [[1556073]]
28   []
29   []
30   []
31   []
32   []
33   []
34   []
35   []
36   []
37   []
38   []
39   []
40   []
41   []
42   []
43   []
44   []
45   []
46   [[1677690]]
47   []
48   []
49   []
50   [[2866901]]
51   [[765072, 2]]
52   []
53   []
54   []
55   []
56   []
57   []
58   []
59   []
60   []
61   []
62   []
63   []
64   []
65   []
66   []
67   []
68   []
69   [[1240414]]
70   [[833646], [1535683]]
71   [[4531], [33402], [36051], [36242], ..., [2889793], [2893992], [2898171]]
72   [[3708], [5639], [6127], [18509], ..., [2883042], [2890813], [2895526]]
73   [[2893], [32363], [34912], [42532], ..., [2881228], [2886162], [2891135]]
74   [[12412], [31415, 2], [36412],

In [83]:
GmtPhi_LS133 = gmt_awk_LS133.gmtPhi
GmtEta_LS133 = gmt_awk_LS133.gmtEta

BmtfEta_LS133 = bmtf_awk_LS133.bmtfEta
BmtfPhi_LS133 = bmtf_awk_LS133.bmtfPhi

GmtEta_LS133 = ak.flatten(ak.where(ak.num(GmtEta_LS133) == 0, ak.Array([[0]]), GmtEta_LS133))
GmtPhi_LS133 = ak.flatten(ak.where(ak.num(GmtPhi_LS133) == 0, ak.Array([[0]]), GmtPhi_LS133))
BmtfEta_LS133 = ak.flatten(ak.where(ak.num(BmtfEta_LS133) == 0, ak.Array([[0]]), BmtfEta_LS133))
BmtfPhi_LS133 = ak.where(ak.num(BmtfPhi_LS133) == 0, ak.Array([[0]]), BmtfPhi_LS133)


In [126]:
print(BX_BMTF_LS133[100, -3])

[2854073, 2]


In [138]:
def DeltaR(BmtfPhi, GmtPhi, BmtfEta, GmtEta):
  return np.sqrt((BmtfPhi - GmtPhi)**2 + (BmtfEta - GmtEta)**2)

def DeltaRDistribution(BX_BMTF, BX_GMT, BmtfPhi, GmtPhi, BmtfEta, GmtEta, use_Bmtf = True):

  DeltaDist = [[] for _ in range(ONE_ORBIT_IN_BX)]
  IndexMin = [[] for _ in range(ONE_ORBIT_IN_BX)]

  if use_Bmtf: 
    for i in range(ONE_ORBIT_IN_BX):

      NestedArrayDeltaR = []
      NestedArrayIndex = []

      for j in range(len(BX_BMTF[i])):
        DeltaRTemp = DeltaR(BmtfPhi[BX_BMTF[i, j, 0]], GmtPhi[BX_GMT[i, :, 0]], BmtfEta[BX_BMTF[i, j, 0]], GmtEta[BX_GMT[i, :, 0]])

        minDeltaR = np.min(DeltaRTemp)
        indexTemp = np.argmin(DeltaRTemp)

        NestedArrayDeltaR.append(minDeltaR)
        NestedArrayIndex.append(indexTemp)

      DeltaDist[i].extend(NestedArrayDeltaR)
      IndexMin[i].extend(NestedArrayIndex)

  return ak.Array(DeltaDist), ak.Array(IndexMin)

In [140]:
#RDistribution = DeltaRDistribution(BX_BMTF_LS133, BX_GMT_LS133, BmtfPhi_LS133, GmtPhi_LS133, BmtfEta_LS133, GmtEta_LS133, use_Bmtf = True)

In [136]:
i = 100
j = -3
print(BmtfPhi_LS133[BX_BMTF_LS133[i, j]])
print(BX_BMTF_LS133[i, j])
print(len(GmtPhi_LS133[BX_GMT_LS133[i, :, 0]]))

[[1.65, 2.45], [0]]
[2854073, 2]
193
